<a href="https://colab.research.google.com/github/i-gitit/webscrape_python/blob/master/stock_change_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install stockstats

In [0]:
#importing libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import stockstats as ss
import time

In [0]:
#Declaring variables
company=[]
lastprice=[]
pcmax=[]  #used two variables to store minimum and max values uf %change so that range of change can be found and compared
pcmin=[]
maxtime=[]    #used to store times when change was max and min so that rate of change could be stated
mintime=[]

#alert function
def alert(c,i):
  if(c):
    str="the stock price of company {} showed an {} percentage increase in value in last {} seconds".format(company[i],pcmax[i]-pcmin[i],maxtime[i]-mintime[i])
    print(str)
    pcmin[i]=pcmax[i]
    mintime[i]=maxtime[i]
  else:
    str="the stock price of company {} showed an {} percentage decrease in value in last {} seconds".format(company[i],pcmax[i]-pcmin[i],mintime[i]-maxtime[i])
    print(str)
    pcmax[i]=pcmin[i]
    maxtime[i]=mintime[i]
  

In [0]:
#fetching html of page 
webdata=requests.get("https://www.moneycontrol.com/stocks/marketstats/indexcomp.php?optex=NSE&opttopic=indexcomp&index=9")
curtime=time.time()
htmldata=webdata.text

soup=BeautifulSoup(htmldata,'lxml')
a = soup.find("table", attrs={'class':'tbldata14 bdrtpg'})

#running loop first time to get initial status of stocks
for tr in a.find_all('tr')[1:]:
    td=tr.find_all('td')
    company.append(td[0].find('b').text)
    lastprice.append(float(td[2].text.replace(',','')))
    pcmin.append(float(td[4].text))
    pcmax.append(float(td[4].text))
    maxtime.append(curtime)
    mintime.append(curtime)

time.sleep(30)

In [0]:
# run the funtion for specific time period and compare %change 
t_end = time.time() + 60*10  #converting minutes into seconds 
while time.time() < t_end:
  
  curtime=time.time()
  i=0 # i is used to index our lists.

  webdata=requests.get("https://www.moneycontrol.com/stocks/marketstats/indexcomp.php?optex=NSE&opttopic=indexcomp&index=9")
  htmldata=webdata.text
  soup=BeautifulSoup(htmldata,'lxml')
    
  a = soup.find("table", attrs={'class':'tbldata14 bdrtpg'})
  
  for tr in a.find_all('tr')[1:]:
    td=tr.find_all('td')
    cur=float(td[4].text) #current value of %change
    
    if(cur>=pcmax[i]):
      maxtime[i]=curtime
      pcmax[i]=cur
      if(pcmax[i]-pcmin[i]>=2):
        alert(1,i)
    
    else:
      if(cur<=pcmax[i]):
        mintime[i]=curtime
        pcmin[i]=cur
        if(pcmax[i]-pcmin[i]>=2):
          alert(0,i)
    i+=1
  
  time.sleep(30)
